In [1]:
import gym
import numpy as np
import tqdm

In [2]:
env = gym.make('FrozenLake-v1')

In [ ]:
env.reset()

In [4]:
n_obs = env.observation_space.n
n_act = env.action_space.n

In [10]:
returns_sum = np.zeros((n_obs, n_act))
returns_count = np.zeros((n_obs, n_act))
max_episodes = 200000
max_iters = 10000
gamma = 1.0
epsilon = 0.3
Q = np.zeros((n_obs, n_act))

In [11]:
def policy(s, Q):
    prob = np.ones(n_act) * epsilon / n_act
    prob[np.argmax(Q[s])] += 1 - epsilon
    a = np.random.choice(np.arange(n_act), p=prob)
    return a

In [ ]:
for i in tqdm.tqdm(range(max_episodes)):
    s = env.reset()
    episode = []
    for i in range(max_iters):
        a = policy(s, Q)
        next_s, r, done, _ = env.step(a)
        episode.append((s, a, r))
        if done:
            break
        s = next_s

    sa_in_episode = set([(i[0], i[1]) for i in episode])
    for s, a in sa_in_episode:
        for i, x in enumerate(episode):
            if x[0] == s and x[1] == a:
                first_idx = i
                break
        G = sum([r * (gamma ** i) for i, (s, a, r) in enumerate(episode[first_idx:])])
        returns_sum[s][a] += G
        returns_count[s][a] += 1
        Q[s][a] = returns_sum[s][a] / (returns_count[s][a] + 1e-10)

In [69]:
reward_list = []
for i in range(100):
    rewards = []
    s = env.reset()
    for i in range(max_iters):
        a = Q[s].argmax()
        next_s, r, done, _ = env.step(a)
        rewards.append(r)
        if done:
            break
        s = next_s
    reward_list.append(rewards)